In [44]:
import functions_ml as fml
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix, make_scorer
import shap
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay, permutation_importance
# Set font properties globally (Times New Roman and font size 14)
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 14
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
results = pd.read_csv('datasets/instances_results.csv')
features = pd.read_csv('datasets/multi_plant_instance_features.csv')
dataset = fml.create_dataset(features, results)

In [46]:
dataset = fml.create_multi_class_target(dataset, tolerance=0.01/100)
dataset['TARGET'] = dataset['TARGET'].apply(lambda x: x[:4]) 
dataset[['TARGET']].value_counts()

TARGET
RF_6      140
RF_2      129
RF_4      117
RF_3      115
RF_1       51
RF_T       46
Name: count, dtype: int64

In [47]:
dataset['TARGET'].apply(lambda x: x[:4])

136    RF_6
459    RF_3
271    RF_6
425    RF_4
452    RF_T
       ... 
57     RF_T
389    RF_4
385    RF_2
166    RF_T
361    RF_2
Name: TARGET, Length: 598, dtype: object

In [48]:
dataset

,num_products,num_plants,num_periods,binary_vars,total_demand,min_demand,avg_demand,max_demand,std_demand,skew_demand,...,std_utilization,skew_utilization,kurt_utilization,cv_utilization,p25_utilization,p50_utilization,p75_utilization,iqr_utilization,instance,TARGET
136,60,2,12,1440,129421,1,89.875694,180,50.912498,0.027624,...,0.068534,-0.296231,-0.652493,0.068516,0.956948,1.004674,1.052507,0.095559,NBA00_12_2_60,RF_6
459,60,15,12,10800,972094,1,90.008704,180,52.141687,0.015824,...,0.063545,-0.037455,-0.777676,0.063585,0.950749,0.994764,1.044994,0.094245,NBA01_12_15_60,RF_3
271,120,2,12,2880,256032,1,88.900000,180,51.558648,0.024490,...,0.056680,-0.353318,-0.704888,0.051026,1.085519,1.121016,1.154742,0.069223,ABB00_12_2_120,RF_6
425,60,4,12,2880,258911,1,89.899653,180,53.218796,0.039713,...,0.061929,0.028792,-0.635130,0.061913,0.959284,0.999527,1.045699,0.086415,NAB02_12_4_60,RF_4
452,120,2,12,2880,256032,1,88.900000,180,51.558648,0.024490,...,0.053999,-0.353292,-0.705290,0.048612,1.086715,1.120573,1.152676,0.065961,AAA00_12_2_120,RF_T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,60,2,12,1440,127578,1,88.595833,180,49.899251,0.059355,...,0.059950,0.080683,-0.966151,0.060042,0.947864,1.005381,1.038224,0.090361,NAB01_12_2_60,RF_T
389,120,6,12,8640,784969,1,90.852894,180,52.076091,-0.010968,...,0.047504,-0.120356,-0.678014,0.042762,1.079601,1.109687,1.153287,0.073686,ABA02_12_6_120,RF_4
385,10,15,12,1800,166837,1,92.687222,180,52.648113,-0.074075,...,0.158498,-0.057660,-0.567183,0.158639,0.884722,1.002411,1.119760,0.235037,NBB04_12_15_10,RF_2
166,120,20,12,28800,2613563,1,90.748715,180,51.989747,-0.015315,...,0.044540,-0.102801,0.133856,0.044542,0.972529,1.001643,1.029890,0.057361,NAA02_12_20_120,RF_T
